<a href="https://colab.research.google.com/github/Lursen/Ensemble-methods/blob/main/Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Класс однослойной нейронной сети прямого распространения

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class FeedForwardNetwork():
  def __init__(self, nIn=51, nHidden=200, nOut=1, iter = 10000):
    # learning rate
    self.alpha = 0.3
    # number of iterations
    self.iter = iter
    # array of errors
    self.errors = []
    self.accuracy = []
    #number of neurons in each layer
    self.nIn = nIn
    self.nHidden = nHidden
    self.nOut = nOut
    # initialize weights randomly (+1 for bias)
    self.hWeights = np.random.randn(self.nHidden, self.nIn+1)
    self.oWeights = np.random.randn(self.nOut, self.nHidden+1)
    # activations of neurons (sum of inputs)
    self.hActivation = np.zeros((self.nHidden, 1), dtype=float)
    self.oActivation = np.zeros((self.nOut, 1), dtype=float)
    # outputs of neurons (after sigmoid function)
    self.iOutput = np.zeros((self.nIn+1, 1), dtype=float) # +1 for bias
    self.hOutput = np.zeros((self.nHidden+1, 1), dtype=float) # +1 for bias
    self.oOutput = np.zeros((self.nOut), dtype=float)
    # deltas for hidden and output layer
    self.hDelta = np.zeros((self.nHidden), dtype=float)
    self.oDelta = np.zeros((self.nOut), dtype=float) 

  def sigmoid(self,Z):
    return 1/(1+np.exp(-Z))

  def forward(self, input):
    # set input as output of first layer (bias neuron = 1.0)
    self.iOutput[:-1, 0] = input
    self.iOutput[-1:, 0] = 1.0
    # hidden layer
    self.hActivation = np.dot(self.hWeights, self.iOutput)
    self.hOutput[:-1, :] = self.sigmoid(self.hActivation)
    # set bias neuron in hidden layer to 1.0
    self.hOutput[-1:, :] = 1.0
    # output layer
    self.oActivation = np.dot(self.oWeights, self.hOutput)
    self.oOutput = self.sigmoid(self.oActivation)

  def get_results(self, teach):
    err =  self.oOutput - teach
    acc = 1 - np.abs(err)
    return err,acc

  def backward(self, teach):
    error =  self.oOutput - teach
    self.errors.append(np.square(error))
    self.accuracy.append(1-np.abs(self.oOutput - teach))
    # deltas of output neurons
    self.oDelta = (1 - self.sigmoid(self.oActivation)) * self.sigmoid(self.oActivation) * error
    # deltas of hidden neurons
    self.hDelta = (1 - self.sigmoid(self.hActivation)) * self.sigmoid(self.hActivation) * np.dot(self.oWeights[:,:-1].transpose(), self.oDelta)
    # apply weight changes
    self.hWeights = self.hWeights - self.alpha * np.dot(self.hDelta, self.iOutput.transpose()) 
    self.oWeights = self.oWeights - self.alpha * np.dot(self.oDelta, self.hOutput.transpose())

  def train(self, X, Y):
    for i in range(0,self.iter):
      rnd = np.random.randint(0,np.shape(X)[0])
      self.forward(X[rnd])
      self.backward(Y[rnd])

  def getErrors(self):
    return self.errors

  def getAccuracy(self):
    return np.mean(self.accuracy)

  def getOutput(self):
    if (self.oOutput >= 0.8):
      return 1
    else:
      return 0


Функции для обучения комитета классификаторов

In [ ]:
import math

def train_commitee(m,x,y,x_test,y_test):
  NN = [[] for i in range(m)]
  accuracy = [[] for i in range(m)]

  for i in range(0,m):
    NN[i] = FeedForwardNetwork()
    NN[i].train(x[i],y[i])

  for t in range(0,50):
    for i in range(0,m):
      rnd = np.random.randint(0,np.shape(x_test)[0])
      NN[i].forward(x_test[rnd])
      err, acc = NN[i].get_results(y_test[rnd])
      accuracy[i].append(acc)

  return NN, accuracy

def get_commitee_answer(NN,X,Y,iter=50):
  accuracy = []
  answer = 0
  for iter in range(0,iter):
    rnd = np.random.randint(0,np.shape(X)[0])
    sum = 0
    for i in range(0,np.shape(NN)[0]):
      NN[i].forward(X[rnd])
      err, acc = NN[i].get_results(Y[rnd])
      sum += NN[i].getOutput()
    #print('rnd:',rnd)
    #print('Sum:',sum)
    #print('Correct answer:', Y[rnd])
    if (sum >= (np.shape(NN)[0])/2 + 1):
      #print('Committee answer:', 1)
      answer = 1
    else:
      #print('Committee answer:', 0)
      answer = 0
    error = np.abs(Y[rnd] - answer)
    accuracy.append(1-error)
  acc_mean = np.mean(accuracy)
  std = np.std(accuracy)
  return acc_mean, std

Загрузка датасета, создание выборок

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#bool_in=0
#real_in=51
#bool_out=2
#real_out=0
#training_examples=345
#validation_examples=173
#test_examples=172

data = np.loadtxt('card2.dt')
X = data[:, 0:51]
Y = data[:, 52]

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.9, random_state=42)

Обучение и тестирование комитетов

5 классификаторов

In [ ]:
import itertools
#number of classifiers
m = 5
n_elements = math.floor(int(X_train.shape[0]/m))

xdt = []
ydt = []

for i in range(0,m):
  fel = i*n_elements
  lel = n_elements*(i+1)
  xdt.append(np.delete(X_train, slice(fel,lel), axis=0))
  ydt.append(np.delete(y_train, slice(fel,lel), axis=0))

NN, accuracy = train_commitee(m,xdt,ydt, X_test, y_test)

In [ ]:
for i in range(0,m):
  print(np.mean(accuracy[i]))

0.8640949390681298
0.8511026965069394
0.8385384119859283
0.7807029228238648
0.8167916546469747


In [ ]:
acc, std = get_commitee_answer(NN,X_test,y_test)
print(acc, std)

0.9 0.3


7 классификаторов

In [ ]:
import itertools
#number of classifiers
m = 7
n_elements = math.floor(int(X_train.shape[0]/m))

xdt = []
ydt = []

for i in range(0,m):
  fel = i*n_elements
  lel = n_elements*(i+1)
  xdt.append(np.delete(X_train, slice(fel,lel), axis=0))
  ydt.append(np.delete(y_train, slice(fel,lel), axis=0))

NN, accuracy = train_commitee(m,xdt,ydt, X_test, y_test)

In [ ]:
for i in range(0,m):
  print(np.mean(accuracy[i]))

0.8931838946810502
0.8241314640757086
0.8424163811407268
0.9123066004855183
0.8459055332427994
0.8617260443993608
0.8866105996403568


In [ ]:
acc, std = get_commitee_answer(NN,X_test,y_test)
print(acc, std)

0.94 0.23748684174075838


11 классификаторов

In [ ]:
import itertools
#number of classifiers
m = 11
n_elements = math.floor(int(X_train.shape[0]/m))

xdt = []
ydt = []

for i in range(0,m):
  fel = i*n_elements
  lel = n_elements*(i+1)
  xdt.append(np.delete(X_train, slice(fel,lel), axis=0))
  ydt.append(np.delete(y_train, slice(fel,lel), axis=0))

NN, accuracy = train_commitee(m,xdt,ydt, X_test, y_test)

In [ ]:
for i in range(0,m):
  print(NN[i].getAccuracy())

0.8582838162764942
0.8390466637435662
0.7683660132865047
0.8729465353117506
0.8590887689233471
0.8564483739229938
0.868153608620625
0.8599903668438738
0.8771143512045999
0.6628491914931516
0.862283924865396


In [ ]:
acc, std = get_commitee_answer(NN,X,Y)
print(acc, std)

0.98 0.13999999999999999
